### 1. 正则的几个用法

1. 匹配手机号

In [1]:
import re

phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')
result = re.findall(phone_p, 'Myphone is :135-0009-4747 or 18847868856')
print(result)

['135-0009-4747', '18847868856']


2.贪婪匹配和非贪婪匹配

- 贪婪匹配: *

- 非贪婪匹配(可能为0个)： *?

- 贪婪匹配所有字符:  .*  

- 非贪婪匹配所有字符: .*? 

In [7]:
import re
p1 = re.compile(r'我爱好.*')
result = re.match(p1, '我爱好好好好')
print(result)

p2 = re.compile(r'我爱好.*?')
result = re.match(p2, '我爱好好好好')
print(result)

<_sre.SRE_Match object; span=(0, 6), match='我爱好好好好'>
<_sre.SRE_Match object; span=(0, 3), match='我爱好'>


3.使用捕获组

括号的使用
findall可以用于找只有一个匹配内容的，相当于只有一个括号的情况，但是当有多个括号的时候，使用findall就会比较麻烦

In [8]:
p = re.compile(r'[a-z]+((\d+)-(\d+))[a-z]+')
 
result = re.match(p, 'afs123-456gds')
print(result.group()) # afs123-456gds
print(result.group(1)) # 123-456
print(result.group(2)) # 123
print(result.group(3)) # 456

afs123-456gds
123-456
123
456


4.字符串的替换

In [9]:
import re
 
p = re.compile(r'\d+')
 
result = re.sub(p, '中文', 'tom32jerry456haha')
print(result) # tom中文jerry中文haha

tom中文jerry中文haha


### 2.构建谷歌搜索函数

- 关键点1:如果实现翻墙？在本地使用了小飞机，如何让我们的python请求也经过小飞机？答案是使用代理，小飞机的代理端口是1080
所以只需要将代理指定到本地的1080即可

- 关键点2:仔细观察谷歌搜索时向服务器发送的请求，可以获得最近一年，最近一个月最近一周的结果


In [8]:
import re
import requests 
from bs4 import BeautifulSoup
proxy = {"http":"http://127.0.0.1:1080","https":"https://127.0.0.1:1080"}
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }


def google(q):
    s = requests.Session()
    q = '+'.join(q.split())
    tbs = '&tbs=qdr:y'
#     url = 'https://www.google.com/search?q=' + q + tbs + '&ie=utf-8&oe=utf-8'
    url = 'https://www.google.com/search?q=' + q + '&start=10&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get,proxies = proxy,verify=False)

    soup = BeautifulSoup(r.text, "html.parser")
    print(soup.find('div', id='resultStats'))
    output = []
#     for searchWrapper in soup.find_all('h3', {'class':'r'}): #this line may change in future based on google's web page structure
#         url = searchWrapper.find('a')["href"] 
#         text = searchWrapper.find('a').text.strip()
#         result = {'text': text, 'url': url}
#         output.append(result)
    
    for content_p in soup.find_all('span', {'class':'st'}):
        text = content_p.text
        result = {'text': text}
        output.append(result)
        print(text)
        print("\n")
    return output

(google("深圳 横岗 发光字 微信"))

C:\Users\mk10730\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<div id="resultStats">Page 2 of about 149,000 results<nobr> (0.25 seconds) </nobr></div>
门头玻璃烤漆外装迷你发光字，与普通亚克力字相比，迷你发光字在体现立体感的 ... 工厂地址：深圳市龙岗区横岗街道西坑社区潭面路8号手机/微信：13728904214 尚 ...


Jun 2, 2018 - 深圳横岗桌椅出租出租各式：发光吧台、大圆桌、玻璃转盘、吧台吧椅、休闲桌、 ... 服务范围：全广东各大小城市联系电话： 陆《微信同步》 欢迎来电洽谈。


深圳-龙岗-横岗 深圳龙岗区横岗 ... 深圳-龙岗区面议 横岗网站、APP、微信开发、微信小程序、商城、上门服务 横岗LED显示屏、门头招牌、金属字发光字水晶字 ...


本公司位于深圳龙岗区橫崗六約工業區，主营 GS挂历|GS台历|GS日历|GS周 ... 手机微信B2C商城开发、响应式设计 公司招牌、形象墙、门头招牌、发光字、灯箱、大型 ...


深圳福恩莱精工字有限公司是一家集专业设计、施工、售后服务为一体的现代化光电艺术制品企业。 ... 地址：深圳市龙岗区横岗街道山塘工业区5号楼 ... 冲孔发光字.


亚克力|有机玻璃|深圳亚克力|深圳亚克力厂家|深圳有机玻璃厂家|亚克力板材| ... QQ:3176702600 电话微信13421805630 韩生 ... 水晶字制作亚克力水晶字批发水晶发光字压克力制品 ... 所在地区 : 广东省深圳市龙岗区横岗街道沿河路41号2栋2楼 ...


微信扫一扫. 快速获取 ... 深圳公司招牌广告制作,公司前台背景墙广告字，形象墙logo标识广告招牌制作。 深圳波 ... 龙华清湖招牌灯箱LED发光字公司形象墙喷绘 1图 ...


May 19, 2017 - 深圳市兴仁达科技有限公司 ... 地址：深圳市龙岗区横岗街道大康大万村水岭路12号 ... 文件下载：树脂发光字培训(点击即可下载) ... 微信公众账号.


精神堡垒 · 导向标识 · 发光字 · 灯箱; 其它. 电话： 0755-89522055 / 18503080246 / 18320960293 邮箱：2936405490@qq.com 地址：深圳市龙岗区横岗188工业区15 ...


2hB广告字雕刻

[{'text': '门头玻璃烤漆外装迷你发光字，与普通亚克力字相比，迷你发光字在体现立体感的 ... 工厂地址：深圳市龙岗区横岗街道西坑社区潭面路8号手机/微信：13728904214 尚\xa0...'},
 {'text': 'Jun 2, 2018 - 深圳横岗桌椅出租出租各式：发光吧台、大圆桌、玻璃转盘、吧台吧椅、休闲桌、 ... 服务范围：全广东各大小城市联系电话： 陆《微信同步》 欢迎来电洽谈。'},
 {'text': '深圳-龙岗-横岗 深圳龙岗区横岗 ... 深圳-龙岗区面议 横岗网站、APP、微信开发、微信小程序、商城、上门服务 横岗LED显示屏、门头招牌、金属字发光字水晶字\xa0...'},
 {'text': '本公司位于深圳龙岗区橫崗六約工業區，主营 GS挂历|GS台历|GS日历|GS周 ... 手机微信B2C商城开发、响应式设计 公司招牌、形象墙、门头招牌、发光字、灯箱、大型\xa0...'},
 {'text': '深圳福恩莱精工字有限公司是一家集专业设计、施工、售后服务为一体的现代化光电艺术制品企业。 ... 地址：深圳市龙岗区横岗街道山塘工业区5号楼 ... 冲孔发光字.'},
 {'text': '亚克力|有机玻璃|深圳亚克力|深圳亚克力厂家|深圳有机玻璃厂家|亚克力板材| ... QQ:3176702600 电话微信13421805630 韩生 ... 水晶字制作亚克力水晶字批发水晶发光字压克力制品 ... 所在地区 : 广东省深圳市龙岗区横岗街道沿河路41号2栋2楼\xa0...'},
 {'text': '微信扫一扫. 快速获取 ... 深圳公司招牌广告制作,公司前台背景墙广告字，形象墙logo标识广告招牌制作。 深圳波 ... 龙华清湖招牌灯箱LED发光字公司形象墙喷绘 1图\xa0...'},
 {'text': 'May 19, 2017 - 深圳市兴仁达科技有限公司 ... 地址：深圳市龙岗区横岗街道大康大万村水岭路12号 ... 文件下载：树脂发光字培训(点击即可下载) ... 微信公众账号.'},
 {'text': '精神堡垒 · 导向标识 · 发光字 · 灯箱; 其它. 电话： 0755-89522055 / 18503080246 / 18320960293 邮箱：2936405490@qq.com 地

In [11]:
#coding:utf-8
import requests
import json
import time
import re

headers = {
    'Host':"map.baidu.com",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:39.0) Gecko/20100101 Firefox/39.0"}

def citys():
    html=requests.get('http://map.baidu.com/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=baidu&pcevaname=pc4.1&qt=s&da_src=searchBox.button&wd=%E6%B1%BD%E8%BD%A6%E7%BE%8E%E5%AE%B9%E5%BA%97&c=1&src=0&wd2=&sug=0&l=5&b=(7002451.220000001,1994587.88;19470675.22,7343963.88)&from=webmap&biz_forward={%22scaler%22:1,%22styles%22:%22pl%22}&sug_forward=&tn=B_NORMAL_MAP&nn=0&u_loc=12736591.152491,3547888.166124&ie=utf-8&t=1459951988807',headers=headers).text
    f=open('city_ids.txt','a')
    data=json.loads(html)
    for item in data['content']:
        #for city in item['city']:
            f.write(str(item)+'\n')
    f.close()

def get_infor(keyword,code,page):
    html=requests.get('http://map.baidu.com/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=baidu&pcevaname=pc4.1&qt=con&from=webmap&c='+str(code)+'&wd='+keyword+'&wd2=&pn='+str(page)+'&nn='+str(page*10)+'&db=0&sug=0&addr=0&&da_src=pcmappg.poi.page&on_gel=1&src=7&gr=3&l=12&tn=B_NORMAL_MAP&u_loc=12736591.152491,3547888.166124&ie=utf-8',headers=headers).text
    data=json.loads(html)['content']
    return data


def main():
    keys=['search']
    for keyword in keys:
        f=open(keyword+'_tels.txt','a')
#         for line in open('city_ids.txt','r').readlines():
#             line=line.replace('\n','')
#             code=340
#             page=1
#             while True:
#                 try:
#                     data=get_infor(keyword,code,page)
#                 except:
#                     break
#                 if data==[]:
#                     break
#                 for item in data:
#                     f.write(str(item['area_name'])+',    ')
#                     f.write(str(item['name'])+',    ')
#                     f.write(str(item['addr'])+',    ')
#                     f.write(str(item['ext']['detail_info']['phone'])+'\n')
#                     # print(str(item['area_name'])+'\n')
                    
#                 page+=1
#                 print(code,page)
#                 time.sleep(1)
#         f.close()
        code=340
        page=1
        while True:
            try:
                data=get_infor(keyword,code,page)
            except:
                break
            if data==[]:
                break
            for item in data:
                f.write(str(item['area_name'])+',    ')
                f.write(str(item['name'])+',    ')
                f.write(str(item['addr'])+',    ')
                f.write(str(item['ext']['detail_info']['phone'])+'\n')
                # print(str(item['area_name'])+'\n')

            page+=1
            print(code,page)
            time.sleep(1)
        f.close()
main()

340 2
340 3
340 4
340 5
340 6
340 7
340 8
340 9
340 10
340 11
340 12
340 13
340 14
340 15
340 16
340 17
340 18
340 19
340 20
340 21
340 22
340 23
340 24
340 25
340 26
340 27
340 28
340 29
340 30
340 31
340 32
340 33
340 34
340 35
340 36
340 37
340 38
340 39
340 40
340 41
340 42
340 43
340 44
340 45
340 46
340 47
340 48
340 49
340 50
340 51
